In [2]:
# function main(args)
#     return args
# end
# Load inline arguments for worker unit
# num_workers, this = main(ARGS)
num_workers = 1 #parse(Int64, num_workers)
this        = 1 #parse(Int64, this)

#
# Load packages and functions
include("../../src/init.jl")
include("../../src/fct.jl")
include("../../src/worker_fct.jl")

#
# Output paths and filter settings
plots_base_path = "../../../waveforms/baselines/plots/raw/"
base_path_raw   = "../../../waveforms/baselines/raw_wf_check/"
base_path       = "../../../waveforms/baselines/wf_check/"

"../../../waveforms/baselines/wf_check/"

In [8]:
calib_filepath = "../../dicts/calib.json"
calib = JSON.parsefile(calib_filepath);

datasets_str = [
    "0053-0064",
    "0065-0079",
    "0080-0092",
    "0093-0113"
]
det_lib = JSON.parsefile("../../dicts/det_lib.json");

In [15]:
ch = 0
ch_str = lpad(ch, 2, "0")
run_str = det_lib[channel_to_name[ch]]["run_str"]
calib[channel_to_name[ch]]["data"][run_str]["used_runs"]

10-element Vector{Any}:
 83
 84
 85
 86
 87
 88
 89
 90
 91
 92

In [24]:
run_strs = glob("../../../waveforms/calib/raw_v07.01/*")
runs = []
for run in run_strs
    isdir(run) ? push!(runs, basename(run)) : ""
end

run_keys  = []
for run in runs
    current_dir = pwd()
    ddir = "/remote/ceph/group/gerda/data/phase2/blind/v07.01/gen"
    cd(joinpath(ddir, "/tier4/all/phy/$run/"))
    files = glob("*.root")
    for meta_key in files
        push!(run_keys, split(meta_key, "-phy-all-tier")[1])
    end 
    cd(current_dir)
end
run_keys_filtered = []
for k in run_keys
    if parse(Int64, split(split(k, "-")[2], "run")[end]) in calib[channel_to_name[ch]]["data"][run_str]["used_runs"]
        push!(run_keys_filtered, k)
    end
end
run_keys = run_keys_filtered;

In [26]:
@info("Run keys are gathered! Number of files: " * string(length(run_keys)))
run_keys = get_share_for_worker(run_keys, num_workers, this)
@info("This worker takes: " * string(length(run_keys)))

┌ Info: Run keys are gathered! Number of files: 2312
└ @ Main In[26]:1
┌ Info: This worker takes: 2312
└ @ Main In[26]:3


In [16]:
for id in run_keys
#     Base.run(`clear`)
    IJulia.clear_output(true)
    int_run = parse(Int64, split(split(id, "-")[2], "run")[end])
    str_run = lpad(int_run, 4, "0");
    ddir = "/remote/ceph/group/gerda/data/phase2/blind/v07.01/gen"
    channels = []
    for ch in 0:1:36
        filename = base_path_raw * "run" * str_run * "/" * lpad(ch, 2, "0") * "-" * channel_to_name[ch] * "/"
        !isdir(filename) ? mkpath(filename) : ""
        filename *= id * "-phy-bl-raw.h5"
        !isfile(filename) ? push!(channels, ch) : ""
    end
    println(length(channels))
    if length(channels) > 0
        filename1 = ddir*"/tier1/ged/phy/run"*str_run*"/"*id*"-phy-ged-tier1.root";
        filename4 = ddir*"/tier4/all/phy/run"*str_run*"/"*id*"-phy-all-tier4.root";
        tree = TFile(filename4)["tier4"];
        current_file_number = findall(x->x == id, run_keys)[1]
        @info("File " * string(current_file_number) * " of " * string(length(run_keys)))
        @info(">---------------------------------------------------------------------<")
        @info("Load waveforms")
        @time events = TypedTables.Table(raw2mgtevent.(TFile(filename1)["MGTree"].event[:]))
        @info("Start selecting & storing")
        waveforms    = events.fAuxWaveforms;
        for ch in channels
            index = findall(x->x[ch + 1] == 0, tree.energy[1:end])
            if length(index) > 0
                energy       = sum.(tree.energy[index])
                multiplicity = tree.multiplicity[index]
                timestamp    = tree.timestamp[index]
                channel      = Int.(zeros(length(index)) .+ ch)

                datasetID    = []
                map(x->push!(datasetID, x[ch+1]), tree.datasetID[index])

                waveform  = [] 
                map(x->push!(waveform, x[ch+1].wf), waveforms[index])

                filtered_dict = Table( 
                    energy       = energy,
                    multiplicity = multiplicity, 
                    timestamp    = timestamp,
                    run          = Int.(zeros(length(index)) .+ int_run),
                    datasetID    = datasetID,
                    channel      = channel, 
                    waveform     = waveform
                )
                events   = nothing;

                filename = base_path_raw * "run" * str_run * "/" * lpad(ch, 2, "0") * "-" * channel_to_name[ch] * "/"
                filename *= id * "-phy-bl-raw.h5"
                HDF5.h5open(filename, "w") do h5f
                    LegendHDF5IO.writedata( h5f, "data", Table( 
                            energy       = float.(filtered_dict.energy),
                            multiplicity = float.(filtered_dict.multiplicity), 
                            timestamp    = float.(filtered_dict.timestamp),
                            run          = Int.(filtered_dict.run),
                            datasetID    = Int.(filtered_dict.datasetID),
                            channel      = Int.(filtered_dict.channel), 
                            waveform     = StructArray{RDWaveform}(Array{RDWaveform,1}(filtered_dict.waveform))
                        )
                    )
                end

                @info("Ch" * lpad(ch, 2, "0") * "-" * channel_to_name[ch] * " | " * string(size(filtered_dict, 1)) * " baselines stored!")
            end
        end
    end
end

37


┌ Info: File 4 of 9496
└ @ Main In[16]:20
┌ Info: >---------------------------------------------------------------------<
└ @ Main In[16]:21
┌ Info: Load waveforms
└ @ Main In[16]:22


  6.821409 seconds (2.43 M allocations: 3.903 GiB, 5.47% gc time)


┌ Info: Start selecting & storing
└ @ Main In[16]:24
┌ Info: Ch00-GD91A | 566 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch01-GD35B | 563 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch02-GD02B | 561 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch03-GD00B | 566 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch04-GD61A | 567 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch05-GD89B | 564 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch06-GD02D | 564 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch07-GD91C | 566 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch08-ANG5 | 553 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch09-RG1 | 1111 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch10-ANG3 | 553 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch11-GD02A | 565 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch12-GD32B | 566 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch13-GD32A | 569 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch14-GD32C | 562 baselines stored!
└ @ Main In[16]:66
┌ Info: Ch15-GD89C |

LoadError: InterruptException:

# Calibrate baselines

In [29]:
calib_filepath = "../../dicts/calib.json"
calib = JSON.parsefile(calib_filepath);

datasets_str = [
    "0053-0064",
    "0065-0079",
    "0080-0092",
    "0093-0113"
]
det_lib = JSON.parsefile("../../dicts/det_lib.json");

In [31]:
# function main(args)
#     return args
# end
# Load inline arguments for worker unit
# num_workers, this = main(ARGS)
num_workers = 1 #parse(Int64, num_workers)
this        = 1 #parse(Int64, this)

#
# Load packages and functions
include("../../src/init.jl")
include("../../src/fct.jl")
include("../../src/worker_fct.jl")
# using Distributed
#
# Output paths and filter settings
plots_base_path = "../../../waveforms/baselines/plots/raw/"
base_path_raw   = "../../../waveforms/baselines/raw_wf/"
base_path       = "../../../waveforms/baselines/wf_check/"
# addprocs(3)
# nprocs()

In [43]:
cal = JSON.parsefile("../../dicts/A_cal.json");
for ch in [0]#0:1:36
    ch_str = lpad(ch,2,"0")
    files = glob(base_path_raw * "*/" * ch_str * "-" * channel_to_name[ch] * "/*.h5");
    
    filtered_files = []
    for file in files
        run_str = split(basename(file), "-")[2]
        run = parse(Int64, split(run_str, "run")[end])
        ds_str = det_lib[channel_to_name[ch]]["run_str"]
        if run in calib[channel_to_name[ch]]["data"][ds_str]["used_runs"]
            push!(filtered_files, file)
        end
    end
    IJulia.clear_output(true)
    @info("Channel Ch" * lpad(ch, 2, "0") * " in progress")
    @showprogress for file in filtered_files
        filename = base_path * split(split(file, "/raw_wf/")[2], "-bl-raw.h5")[1] * ".h5"
        run_str = split(basename(file), "-")[2]
        run = parse(Int64, split(run_str, "run")[end])
        if haskey(cal, channel_to_name[ch])
            if haskey(cal[channel_to_name[ch]], run_str) && !isfile(filename)
                data = HDF5.h5open(file, "r") do h5f
                    LegendHDF5IO.readdata(h5f, "data")
                end
                cal_pulses = []
                map(x->push!(cal_pulses, RDWaveform(x.time, (float.(x.value) .- mean(x.value)) ./ cal[channel_to_name[ch]][run_str][1] )), data.waveform);

                !isdir(dirname(filename)) ? mkpath(dirname(filename)) : ""
                HDF5.h5open(filename, "w") do h5f
                    LegendHDF5IO.writedata(h5f, "data", Table(
                            energy = data.energy,
                            multiplicity = data.multiplicity,
                            timestamp = data.timestamp,
                            run = data.run,
                            datasetID = data.datasetID,
                            channel = data.channel,
                            waveform = StructArray{RDWaveform}(Array{RDWaveform,1}(cal_pulses))
                        )
                    )
                end
            end
        end
    end
end

┌ Info: Channel Ch00 in progress
└ @ Main In[43]:16
